In [1]:
import os
os.environ["OMP_NUM_THREADS"]="1"
import random
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision.transforms import transforms
import numpy as np
import toml
from time import time
import segmentation_models_pytorch as smp
from torchvision.transforms.transforms import RandomApply, GaussianBlur, ColorJitter
from src.data_utils import GaussianNoise
from src.color_conversion import Rgb2Hed, Hed2Rgb, LinearContrast
from src.embedding_loss import SpatialEmbLoss
from src.focal_loss import FocalCE
from src.unet import UNet
from src.multi_head_unet import UnetDecoder, MultiHeadModel
from src.spatial_augmenter import SpatialAugmenter
from src.train_utils import supervised_train_step, validation, save_snapshot, save_model, make_cpvs
from src.data_utils import SliceDataset, CropDataset, color_augmentations, add_3c_gt
#import torch_optimizer as optim

torch.backends.cudnn.benchmark = True

torch.manual_seed(42)

params = {
    'data_path': 'tiles',
    'experiment' : 'uniform_MH_attention_finetune',
    'batch_size': 4,
    'training_steps':400000,
    'in_channels': 3,
    'num_fmaps': 32,
    'fmap_inc_factors': 2,
    'downsample_factors': [ [ 2, 2,], [ 2, 2,], [ 2, 2,], [ 2, 2,],],
    'num_fmaps_out': 12,
    'constant_upsample': False,
    'padding': 'same',
    'activation': 'ReLU',
    'weight_decay': 1e-4,
    'learning_rate': 1e-4,
    'seed': 42,
    'num_validation': 500,
    'cutout_prob':0.0,
    'checkpoint_path': 'uniform_MH_attention/train/best_model',
    'cutout_or_RandomErasing': 'RandomErasing',
    'pretrained_model': True,
    'multi_head': True,
    'uniform_class_sampling': True,
    'optimizer': 'AdamW', # one of SGD AdamW AdaBound , Adahessian breaks memory and is not supported
    'validation_step' : 500,
    'snapshot_step' : 5000,
    'checkpoint_step': 20000,
    'instance_seg': 'cpv_3c', # 'embedding' or 'cpv_3c'
    'attention': True
    }

aug_params_fast = {
    'mirror': {'prob_x': 0.5, 'prob_y': 0.5, 'prob': 0.5},
    'translate': {'max_percent':0.05, 'prob': 0.2},
    'scale': {'min': 0.8, 'max':1.2, 'prob': 0.2},
    'zoom': {'min': 0.8, 'max':1.2, 'prob': 0.2},
    'rotate': {'max_degree': 179, 'prob': 0.75},
    'shear': {'max_percent': 0.1, 'prob': 0.2},
    'elastic': {'alpha': [120,120], 'sigma': 8, 'prob': 0.5}
}

log_dir = os.path.join(params['experiment'],'train')
snap_dir = os.path.join(log_dir,'snaps')
os.makedirs(snap_dir,exist_ok=True)
writer_dir = os.path.join(log_dir,'summary',str(time()))
os.makedirs(writer_dir,exist_ok=True)
writer = SummaryWriter(writer_dir)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
params['device'] = device
params['aug_params_fast'] = aug_params_fast
with open(os.path.join(params['experiment'], 'params.toml'), 'w') as f:
    toml.dump(params, f)

if 'tiles' in params['data_path']:
    img_data =  np.load(os.path.join(params['data_path'], 'images.npy')) # N x H x W x C
    lab_dat = np.load(os.path.join(params['data_path'], 'labels.npy')) # N x H x W x 2
    np.random.seed(params['seed'])
    seed_ind = np.random.permutation(img_data.shape[0])
    img_data = img_data[seed_ind]
    lab_dat = lab_dat[seed_ind]
    X_train = img_data[:-params['num_validation']]
    Y_train = lab_dat[:-params['num_validation']]
    X_val = img_data[-params['num_validation']:]
    Y_val = lab_dat[-params['num_validation']:]
    # add gt_3c
    if params['instance_seg'] == 'cpv_3c':
        print('Add 3c labels')
        Y_train = add_3c_gt(Y_train)
        Y_val = add_3c_gt(Y_val)
    labeled_dataset = SliceDataset(raw=X_train, labels=Y_train)
    validation_dataset = SliceDataset(raw=X_val, labels=Y_val)
else:
    file_paths = [os.path.join(params['data_path'], file) for file in os.listdir(params['data_path'])]
    random.Random(params['seed']).shuffle(file_paths)
    val_paths = [file_paths.pop(0) for _ in range(params['num_validation'])]
    labeled_paths = file_paths
    labeled_dataset = CropDataset(labeled_paths, raw_keys=['raw'], label_keys=['gt_labels', 'gt_instances'], crop_size=[256,256])
    validation_dataset = CropDataset(val_paths, raw_keys=['raw'], label_keys=['gt_labels', 'gt_instances'], crop_size=[256,256])

if params['uniform_class_sampling']:
    print('Uniform class sampling')
    # uniform class sampling
    labeled_dataloader = DataLoader(labeled_dataset,
                        batch_size=1,
                        shuffle=False,
                        prefetch_factor=4,
                        num_workers=6)
    classes = [0,1,2,3,4,5,6]
    count_list = []

    for raw, gt in labeled_dataloader:
        gt_classes = gt[...,1].squeeze()
        tmp_list = []
        for c in classes:
            tmp_list.append(torch.sum(gt_classes==c)) # sum of individual classes for a sample
        count_list.append(torch.stack(tmp_list)) # 
        
    counts = torch.stack(count_list) # n_samples x classes 
    sampling_weights = counts/counts.sum(0).unsqueeze(0) # n_samples x classes / 1 x classes = n_samples x classes
    sampling_weights = sampling_weights.sum(1) # n_samples
    sampler = torch.utils.data.WeightedRandomSampler(sampling_weights, num_samples=len(sampling_weights), replacement=True)

    labeled_dataloader = DataLoader(labeled_dataset,
                        batch_size=1,
                        prefetch_factor=4,
                        sampler=sampler,
                        num_workers=6)
else:
    labeled_dataloader = DataLoader(labeled_dataset,
                        batch_size=params['batch_size'],
                        shuffle=True,
                        prefetch_factor=4,
                        num_workers=6)

validation_dataloader = DataLoader(validation_dataset,
                    batch_size=1,
                    shuffle=True,
                    prefetch_factor=4,
                    num_workers=1)

if params['pretrained_model']:
    if params['multi_head']:
        encoder = smp.encoders.get_encoder(
            name= "timm-efficientnet-b5",
            in_channels=3,
            depth=5,
            weights="imagenet").to(device)
        decoder_channels = (256, 128, 64, 32, 16)
        decoder_inst = UnetDecoder(
                    encoder_channels=encoder.out_channels,
                    decoder_channels=decoder_channels,
                    n_blocks=5,
                    use_batchnorm=True,
                    center=False,
                    attention_type='scse' if params['attention'] else None
                    ).to(device)
        decoder_ct = UnetDecoder(
                    encoder_channels=encoder.out_channels,
                    decoder_channels=decoder_channels,
                    n_blocks=5,
                    use_batchnorm=True,
                    center=False,
                    attention_type='scse' if params['attention'] else None
                    ).to(device)
        head_inst = smp.base.SegmentationHead(
                    in_channels=decoder_channels[-1],
                    out_channels=5,
                    activation=None,
                    kernel_size=1).to(device)
        head_ct = smp.base.SegmentationHead(
                    in_channels=decoder_channels[-1],
                    out_channels=7,
                    activation=None,
                    kernel_size=1).to(device)

        decoders = [decoder_inst, decoder_ct]
        heads = [head_inst, head_ct]
        model = MultiHeadModel(encoder, decoders, heads)
    else:
        model = smp.Unet(
            encoder_name= "timm-efficientnet-b5", # "timm-efficientnet-b5", # choose encoder
            encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
            in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
            classes=params['num_fmaps_out'],                      # model output channels (number of classes in your dataset)
            ).to(params['device'])
#preprocess_input = smp.encoders.get_preprocessing_fn('efficientnet-b5', pretrained='imagenet')
else:
    model = UNet(in_channels = params['in_channels'],
                num_fmaps = params['num_fmaps'],
                fmap_inc_factor = params['fmap_inc_factors'],
                downsample_factors = params['downsample_factors'],
                activation = params['activation'],
                padding = params['padding'],
                num_fmaps_out = params['num_fmaps_out'],
                constant_upsample = params['constant_upsample'],
            ).to(params['device'])

if 'checkpoint_path' in params.keys() and params['checkpoint_path']:
    model.load_state_dict(torch.load(params['checkpoint_path'])['model_state_dict'])
model = model.train()

if 'cutout_or_RandomErasing' in params.keys() and params['cutout_or_RandomErasing'] == 'RandomErasing':
    eraser = torch.nn.Sequential(
        *[transforms.RandomErasing(p=1., scale=(0.001, 0.001), ratio=(0.3,1.3), inplace=False) for _ in range(200)])
    def cutout(img):
        B,C,_,_ = img.shape
        device = img.device
        img_list = []
        mask_list = []
        for b in range(B):
            mask = eraser(torch.ones_like(img[b:b+1,...]).to(device))
            img_masked = img[b:b+1,...] * mask.float()
            mask = -1 * (mask-1)
            img_list.append(img_masked) 
            mask_list.append(mask)
        return torch.cat(img_list, axis=0), torch.cat(mask_list, axis=0)

# initialize optimizer, augmentations and summary writer
if params['optimizer'] == 'SGD':
    optimizer = torch.optim.SGD(model.parameters(),
                                lr=params['learning_rate'],
                                momentum=0.9,
                                weight_decay=params['weight_decay'],
                                nesterov=True)
elif params['optimizer'] == 'AdamW':
    optimizer = torch.optim.AdamW(model.parameters(),
                                  lr=params['learning_rate'],
                                  weight_decay=params['weight_decay'])

# elif params['optimizer'] == 'AdaBound':
#     optimizer = optim.AdaBound(model.parameters(),
#                                lr=params['learning_rate'],
#                                final_lr = 1e-5,
#                                weight_decay=params['weight_decay'])

#lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=params['training_steps'], eta_min=1e-5)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=100)
fast_aug = SpatialAugmenter(aug_params_fast)#, padding_mode='reflection')
if params['instance_seg'] == 'embedding':
    inst_loss_fn = SpatialEmbLoss(n_sigma=2, to_center=False, foreground_weight=10, H=256,W=256).to(device)
elif params['instance_seg'] == 'cpv_3c':
    def inst_loss_fn(input, gt_inst, gt_3c):
        gt_cpv_list = []
        gt_inst = gt_inst.squeeze(0)
        for b in range(gt_inst.shape[0]):
            gt_cpv_list.append(make_cpvs(gt_inst.to(device)[b], device))
        gt_cpv = torch.cat(gt_cpv_list, axis=0)
        loss_cpv = F.mse_loss(input = input[:,:2],target = gt_cpv.float())
        loss_3c =  F.cross_entropy(input = input[:,2:],
                        target = gt_3c.long().squeeze(0).to(device),
                        weight=torch.tensor([1,1,2]).float().to(device))
        return loss_cpv + loss_3c

ce_loss_fn = FocalCE(num_classes=7)

def color_augmentations(size, s=0.2):
    # taken from https://github.com/sthalles/SimCLR/blob/master/data_aug/contrastive_learning_dataset.py
    """Return a set of data augmentation transformations as described in the SimCLR paper."""
    color_jitter = ColorJitter(0.8 * s, 0.0 * s, 0.8 * s, 0.2 * s) # brightness, contrast, saturation, hue
    HED_contrast = torch.nn.Sequential(
                Rgb2Hed(),
                LinearContrast(alpha=(0.75,1.25)),
                Hed2Rgb())
    data_transforms = torch.nn.Sequential(
        RandomApply([
            HED_contrast,
            color_jitter,
            GaussianNoise(0.005), 
            GaussianBlur(kernel_size=3, sigma=(0.1,0.1))], p=0.5),
        )
    return data_transforms

color_aug_fn = color_augmentations(200, s=0.2)
validation_loss = []
step = -1

def supervised_training(params):
    step = -1
    while step < params['training_steps']:
        tmp_loader = iter(labeled_dataloader)
        for raw, gt in tmp_loader:
            step += 1
            optimizer.zero_grad()
            loss, pred_inst, pred_class ,img_caug, gt_inst, gt_ct, cutout_maps = supervised_train_step(model,
                                                                raw,
                                                                gt,
                                                                fast_aug,
                                                                color_aug_fn,
                                                                cutout,
                                                                params['cutout_prob'],
                                                                inst_loss_fn,
                                                                ce_loss_fn,
                                                                writer,
                                                                device,
                                                                step)
            loss.backward()
            optimizer.step()
            lr_scheduler.step(loss)
            if step % params['validation_step'] == 0:
                val_new = validation(model,
                        validation_dataloader,
                        inst_loss_fn,
                        ce_loss_fn,
                        device,
                        step,
                        writer)
                validation_loss.append(val_new)
                if val_new <= np.min(validation_loss):
                    print('Save best model')
                    save_model(step, model, optimizer, loss, os.path.join(log_dir,"best_model"))
            #
            if step % params['snapshot_step'] == 0:
                print('Save snapshot')
                tmp_dic = {
                        'pred_ct': pred_class[0].cpu().detach().numpy(),
                        'img_caug': img_caug[0].squeeze(0).cpu().detach().numpy(),
                        'gt_inst': gt_inst.squeeze(0).cpu().detach().numpy()[:1],
                        'gt_ct': gt_ct.squeeze(0).cpu().detach().numpy()[:1]}
                if params['instance_seg'] == 'embedding':
                    _,_,h,w = pred_inst.shape
                    xym_s = inst_loss_fn.xym[:, 0:h, 0:w].contiguous()
                    spatial_emb = pred_inst[0, 0:2] + xym_s  # 2 x h x w
                    sigma = pred_inst[0, 2:2+inst_loss_fn.n_sigma]  # n_sigma x h x w
                    seed_map = torch.sigmoid(
                        pred_inst[0, 2+inst_loss_fn.n_sigma:2+inst_loss_fn.n_sigma + 1])  # 1 x h x w
                    tmp_dic['embedding'] = spatial_emb.cpu().detach().numpy()
                    tmp_dic['sigma'] = sigma.cpu().detach().numpy()
                    tmp_dic['seed_map'] = seed_map.cpu().detach().numpy()
                elif params['instance_seg'] == 'cpv_3c':
                    tmp_dic['pred_cpv'] = pred_inst[0,:2].cpu().detach().numpy()
                    tmp_dic['pred_3c'] = pred_inst[0,2:].softmax(0).cpu().detach().numpy()
                save_snapshot(snap_dir, tmp_dic, step)
            if step % params['checkpoint_step'] == 0:
                save_model(step, model, optimizer, loss, os.path.join(log_dir,"checkpoint_step_"+str(step)))    

cuda:0
Add 3c labels


  0%|          | 0/4481 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

Uniform class sampling


In [ ]:
supervised_training(params)

loss 0.009674789384007454
Validation loss:  5.143734455108643
Save snapshot
Save training snapshot
loss 2.207498073577881
loss 2.1298484802246094
loss 2.4535906314849854
loss 2.3097970485687256
loss 2.505545139312744
loss 1.7809875011444092
loss 3.9863412380218506
loss 3.940455913543701
loss 1.8114243745803833
loss 2.0088958740234375
loss 2.8526899814605713
loss 2.209479331970215
loss 1.919722557067871
loss 1.9440218210220337
loss 2.132370948791504
loss 2.162086009979248
loss 1.9799443483352661
loss 1.9538328647613525
loss 2.2774486541748047
loss 1.8322738409042358
loss 1.8791717290878296
loss 1.7649509906768799
loss 2.0316922664642334
loss 2.314291477203369
loss 2.2840723991394043
loss 2.2823586463928223
loss 1.9057159423828125
loss 2.3778128623962402
loss 6.543913841247559
loss 2.4401376247406006
loss 1.8943179845809937
loss 2.258263111114502
loss 1.6787946224212646
loss 1.938812255859375
loss 2.3986220359802246
loss 2.2491421699523926
loss 1.8761630058288574
loss 1.8250497579574585


loss 1.9761805534362793
loss 1.983403205871582
loss 3.006953716278076
loss 4.060281753540039
loss 2.073866844177246
loss 2.0524282455444336
loss 4.4483232498168945
loss 3.097970962524414
loss 2.5460824966430664
loss 1.631773829460144
loss 3.201465368270874
loss 2.1785614490509033
loss 1.7721296548843384
loss 2.5647382736206055
loss 2.1547257900238037
loss 2.0015757083892822
loss 1.688418984413147
loss 1.5722215175628662
loss 3.879628896713257
loss 4.908225059509277
loss 2.553891658782959
loss 1.882859230041504
loss 1.815675973892212
loss 1.7439556121826172
loss 1.805985927581787
loss 2.276280641555786
loss 1.577072024345398
loss 2.006838083267212
loss 2.2858378887176514
loss 2.2053961753845215
loss 2.2494988441467285
loss 1.9489277601242065
loss 2.3075132369995117
loss 1.6863086223602295
loss 1.9601430892944336
loss 2.011688470840454
loss 1.7203774452209473
loss 3.016411304473877
loss 1.7643043994903564
loss 1.8377293348312378
loss 1.684484839439392
loss 2.459214210510254
loss 2.260727

loss 3.636446475982666
loss 1.6185566186904907
loss 2.1683924198150635
loss 2.263603925704956
loss 3.041222333908081
loss 2.3214235305786133
loss 1.8346731662750244
loss 3.842499256134033
loss 2.1010918617248535
loss 2.598259210586548
loss 2.582963466644287
loss 4.976944446563721
loss 2.051158905029297
loss 3.413689136505127
loss 1.8075246810913086
loss 1.7786736488342285
loss 1.9990633726119995
loss 2.7097678184509277
loss 2.153632164001465
loss 1.779979944229126
loss 2.8629367351531982
loss 5.133172988891602
loss 2.184199810028076
loss 2.966343879699707
loss 1.905975580215454
loss 4.603979110717773
loss 2.0345473289489746
loss 1.9336848258972168
loss 2.156851291656494
loss 1.7947248220443726
loss 3.56017804145813
loss 1.703438401222229
loss 1.9815118312835693
loss 1.9573719501495361
loss 2.0753793716430664
loss 2.676752805709839
loss 1.960186243057251
loss 1.5889655351638794
loss 4.042526721954346
loss 1.757025957107544
loss 2.2426204681396484
loss 2.238124370574951
loss 1.8434214591

loss 2.001682758331299
loss 2.2454922199249268
loss 1.9289610385894775
loss 2.4026474952697754
loss 1.7521920204162598
loss 2.0412001609802246
loss 2.070094585418701
loss 1.9278061389923096
loss 1.776104211807251
loss 1.917750597000122
loss 1.9274494647979736
loss 2.3877511024475098
loss 1.8430335521697998
loss 2.6678757667541504
loss 4.623200416564941
loss 1.776991367340088
loss 1.8570246696472168
loss 2.862428903579712
loss 2.004718065261841
loss 1.9242181777954102
loss 1.680991768836975
loss 2.499232292175293
loss 2.2932021617889404
loss 2.291433811187744
loss 2.5751771926879883
loss 2.0364432334899902
loss 1.669705867767334
loss 2.5916595458984375
loss 2.070920467376709
loss 2.42073917388916
loss 2.6633782386779785
loss 1.9717261791229248
loss 1.7466113567352295
loss 3.1054301261901855
loss 2.4721477031707764
loss 2.74057936668396
loss 1.8531215190887451
loss 3.069119453430176
loss 1.8568196296691895
loss 2.3320553302764893
loss 2.6597862243652344
loss 3.389547824859619
loss 1.5941

loss 2.7502121925354004
loss 2.2027220726013184
loss 2.0178277492523193
loss 1.8948416709899902
loss 1.7156518697738647
loss 2.508941411972046
loss 2.1484427452087402
loss 2.1434407234191895
loss 2.1454179286956787
loss 2.1402573585510254
loss 1.8053944110870361
loss 1.7971446514129639
loss 2.34179425239563
loss 2.282707452774048
loss 2.125014305114746
loss 3.311598300933838
loss 1.5442959070205688
loss 1.7187538146972656
loss 1.783675193786621
loss 1.8850188255310059
loss 1.8796169757843018
loss 2.5578973293304443
loss 1.9679157733917236
loss 1.942244291305542
loss 2.6188735961914062
loss 1.8030235767364502
loss 3.6719775199890137
loss 2.576817035675049
loss 1.8433692455291748
loss 1.9253439903259277
loss 1.9772168397903442
loss 2.3792123794555664
loss 3.2987306118011475
loss 8.72019100189209
loss 2.156562089920044
loss 4.256793975830078
loss 1.9603602886199951
loss 2.1873059272766113
loss 1.7741260528564453
loss 1.8246197700500488
loss 2.5381579399108887
loss 4.294504642486572
loss 1

...

loss 2.6368534564971924
loss 2.155662775039673
loss 2.633291006088257
loss 2.2753968238830566
loss 2.0935652256011963
loss 2.9213943481445312
loss 1.9419586658477783
loss 2.2652931213378906
loss 1.929648518562317
loss 2.4249801635742188
loss 2.004051685333252
loss 1.8450465202331543
loss 2.3206567764282227
loss 3.125143051147461
loss 3.0334668159484863
loss 2.082340717315674
loss 2.1431777477264404
loss 2.1437954902648926
loss 2.941779136657715
loss 2.5013303756713867
loss 3.8839564323425293
loss 1.8187611103057861
loss 2.505388021469116
loss 2.02996826171875
loss 2.0432071685791016
loss 1.8679258823394775
loss 1.9695165157318115
loss 2.5775794982910156
loss 3.502633571624756
loss 3.089223861694336
loss 1.679245948791504
loss 2.0100269317626953
loss 1.8679745197296143
loss 2.1942291259765625
loss 1.8320821523666382
loss 2.0158376693725586
loss 2.5658740997314453
loss 2.97281551361084
loss 2.5663869380950928
loss 2.722529411315918
loss 1.66225004196167
loss 7.4421706199646
loss 1.851427

loss 1.699782133102417
loss 1.898322343826294
loss 1.908558964729309
loss 2.2801568508148193
loss 2.154980421066284
loss 2.4633100032806396
loss 2.7261319160461426
loss 1.7167426347732544
loss 1.7898871898651123
loss 1.9462037086486816
loss 2.5315146446228027
loss 4.63529634475708
loss 2.417147636413574
loss 1.7301437854766846
loss 1.850219964981079
loss 2.3178536891937256
loss 1.8536661863327026
loss 1.9080100059509277
loss 2.4715213775634766
loss 3.882145643234253
loss 3.813030958175659
loss 2.432377815246582
loss 2.3327348232269287
loss 2.165969133377075
loss 2.757774829864502
loss 2.745770215988159
loss 1.7491183280944824
loss 1.9380254745483398
loss 2.1978759765625
loss 1.755276083946228
loss 2.602969169616699
loss 1.803386926651001
loss 1.9935269355773926
loss 1.710628867149353
loss 3.321915626525879
loss 2.3974642753601074
loss 2.0157229900360107
loss 2.0003507137298584
loss 2.3332135677337646
loss 2.1543989181518555
loss 2.0518927574157715
loss 2.324148178100586
loss 1.83622097

loss 1.812551736831665
loss 4.334601879119873
loss 2.0612030029296875
loss 2.876804828643799
loss 2.3072586059570312
loss 1.9956928491592407
loss 2.241908073425293
loss 2.3669772148132324
loss 3.0434796810150146
loss 2.013634443283081
loss 2.1185390949249268
loss 2.2244091033935547
loss 3.208902359008789
loss 1.9388182163238525
loss 1.6099917888641357
loss 2.631622791290283
loss 1.8990283012390137
loss 2.0226807594299316
loss 2.2084872722625732
loss 2.39383602142334
loss 2.618382453918457
loss 3.054318904876709
loss 1.785091519355774
loss 2.5422115325927734
loss 3.1938908100128174
loss 1.7727735042572021
loss 6.503715515136719
loss 1.7846202850341797
loss 6.0482096672058105
loss 2.3064839839935303
loss 2.691439151763916
loss 5.308753967285156
loss 3.131589889526367
loss 7.092771053314209
loss 1.8539087772369385
loss 9.897679328918457
loss 1.6436625719070435
loss 2.052661895751953
loss 1.8932833671569824
loss 4.191561222076416
loss 1.699704885482788
loss 1.8443044424057007
loss 1.923726

loss 1.6915876865386963
loss 7.216787338256836
loss 1.7257710695266724
loss 2.3390164375305176
loss 1.8747624158859253
loss 2.072887897491455
loss 2.572263717651367
loss 1.8637704849243164
loss 1.8897396326065063
loss 1.8685768842697144
loss 3.411329507827759
loss 6.398367404937744
loss 2.4866790771484375
loss 2.0585060119628906
loss 6.392102241516113
loss 1.9143894910812378
loss 1.7870019674301147
loss 1.814809799194336
loss 2.172015428543091
loss 1.879676342010498
loss 18.029308319091797
loss 1.821503758430481
loss 2.067897081375122
loss 2.4817254543304443
loss 1.8144268989562988
loss 2.0931835174560547
loss 3.4970316886901855
loss 2.279068946838379
loss 2.4376540184020996
loss 3.0589070320129395
loss 2.0553817749023438
loss 1.8261075019836426
loss 2.0440316200256348
loss 2.2359941005706787
loss 1.9629175662994385
loss 1.781083106994629
loss 3.333599090576172
loss 1.7595727443695068
loss 2.890226125717163
loss 1.7645156383514404
loss 2.152149200439453
loss 2.6567397117614746
loss 3.4

loss 2.603043556213379
loss 2.1205592155456543
loss 2.1729373931884766
loss 1.899111270904541
loss 2.2464709281921387
loss 1.9848042726516724
loss 3.384086847305298
loss 1.809814453125
loss 4.578028678894043
loss 2.9922285079956055
loss 36.93474578857422
loss 2.668962001800537
loss 3.290506601333618
loss 2.8881845474243164
loss 2.111605167388916
loss 1.7547732591629028
loss 1.9522732496261597
loss 1.901556134223938
loss 0.01694295182824135
loss 1.920662760734558
loss 2.5715627670288086
loss 2.299560785293579
loss 2.148627758026123
loss 2.5665171146392822
loss 1.8967137336730957
loss 2.2248919010162354
loss 1.9110162258148193
loss 2.8365817070007324
loss 1.9850776195526123
loss 1.9336738586425781
loss 1.8209114074707031
loss 2.62797212600708
loss 3.12886643409729
loss 3.2551653385162354
loss 1.8418693542480469
loss 1.9178001880645752
loss 2.1404476165771484
loss 2.1591930389404297
loss 1.7178748846054077
loss 1.578614592552185
loss 2.4109511375427246
loss 1.9708266258239746
loss 3.09947